In [2]:
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad

import iqplot
from scipy import io
from scipy.io import mmread
from scipy.optimize import curve_fit
from scipy.stats import ttest_ind

import sys
import warnings

#plotting 
import iqplot

from bokeh.layouts import column, row
import bokeh.io

#bokeh export set up
from selenium import webdriver
import chromedriver_binary  # Adds chromedriver binary to path

driver = webdriver.Chrome()
driver.get("http://www.python.org")
assert "Python" in driver.title
from bokeh.io import export_png

bokeh.io.output_notebook()

The chromedriver version (131.0.6778.85) detected in PATH at C:\Users\cpuki\anaconda3\Lib\site-packages\chromedriver_binary\chromedriver.exe might not be compatible with the detected chrome version (132.0.6834.197); currently, chromedriver 132.0.6834.159 is recommended for chrome 132.*, so it is advised to delete the driver in PATH and retry


Loading BokehJS ...

# Supplementary Figure 3: Mean Variance Plot Analysis

Producing two layouts for supplementary figure 3.


In [4]:
#load in the data 
file_name = "Data_Files/Filtered_NoContam_annotated_data_10082024.h5ad"
data = ad.read_h5ad(file_name)

In [87]:
#remove mouse with 3 cells
data = data[data.obs["Mouse"] != 4788.0]

def curve_fitted(x,a,c):
    return a*x**2 + c

ASO_plts = []
WT_plts = []
fit_values = []
for mouse in data.obs["Mouse"].unique():
    data_mouse = data[data.obs["Mouse"] == mouse] #segment the data into each mouse
    geno = data_mouse.obs["Genotype"].iloc[0]

    #converting to an array and computing mean and variances
    data_array = data_mouse.X.toarray()
    mean = np.mean(data_array,axis=0)
    variance = np.var(data_array,axis=0)
    
    #curve fit
    popt, _ = curve_fit(curve_fitted,mean,variance,p0 = [0.5,0])
    fit_values.append((mouse,geno,popt[0],popt[1]))

    #produce the variance mean plots
    p = bokeh.plotting.figure(height = 400, width = 600,
                             #title = "Mouse " + str(mouse) + ", a = " + str(np.round(popt[0], decimals=4, out=None)),
                              y_range = (5*10**-3,10**5),
                              # x_axis_label = "Mean", y_axis_label = "Variance",
                              x_axis_type ="log",y_axis_type='log'
                             )
    #remove the toolbar
    p.toolbar.logo = None
    p.toolbar_location = None

    #changing text size
    p.axis.major_label_text_font_size = '34px'
    
    #plotting the data
    p.scatter(mean, variance,color='black',alpha=0.2)
    
    #plot the fit curve
    mean_fit = np.linspace(10**-2,max(mean),1000)
    var_fit = curve_fitted(mean_fit,*popt)
    p.line(mean_fit, var_fit,color='red',alpha=1)
    
    #seggregate plots into ASO or WT columns
    if geno == 'ASO':
        ASO_plts.append(p)
    else:
        WT_plts.append(p)

    

bokeh.io.show(row(column(WT_plts),column(ASO_plts)))

Exporting the Layout

In [55]:
plots = row(column(WT_plts),column(ASO_plts))
export_png(plots, filename="Fits.png")

'C:\\Users\\cpuki\\SnRNAseq_Analysis_Revised\\Fits.png'

Creating Plots of the fit vlaues and running a T-test on each set of data

In [83]:
#plot of fit values
df = pd.DataFrame(fit_values, columns = ["Mouse", "Genotype","a","c"])

#iqplot
colors = ['#00bfc4','#f8766d']
p = iqplot.strip(
    data=df,q = 'a',cats = ["Genotype"],
    q_axis ='y',palette  = colors,title = '', width = 300, height = 400,marker_kwargs = dict(size =8)
)

q = iqplot.strip(
    data=df,q = 'c',cats = ["Genotype"],
    q_axis ='y',palette  = colors,title = '', width = 300, height = 400,marker_kwargs = dict(size =8)
)
#remove the toolbars
p.toolbar.logo = None
p.toolbar_location = None

q.toolbar.logo = None
q.toolbar_location = None

#change font sizes
size = '28px'
p.axis.major_label_text_font_size = size
p.axis.axis_label_text_font_size = size
q.axis.major_label_text_font_size = size
q.axis.axis_label_text_font_size = size


#show the layout
bokeh.io.show(row(p,q))

#Save the layout
plots = row(row(p,q))
export_png(plots, filename="Fit_values.png")

#T-test 
group1 = df[df['Genotype']=='WT']
group2 = df[df['Genotype']=='ASO']

#perform independent two sample t-test
print("results for a", ttest_ind(group1['a'], group2['a'],equal_var = False))
print("results for c", ttest_ind(group1['c'], group2['c'],equal_var = False))

results for a TtestResult(statistic=0.44862785789400583, pvalue=0.6675931379796196, df=6.8267632203828414)
results for c TtestResult(statistic=1.0109812492501795, pvalue=0.3474629181027753, df=6.6351204771718555)
